## Monthly Accounting

In [1]:
import os
from bs4 import BeautifulSoup
import requests
import pymongo
import re

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

service = Service(executable_path='r/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=options)

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = "https://www.khanacademy.org/math/probability/xa88397b6:display-quantitative/xa88397b6:frequency-tables-dot-plots/v/frequency-tables-and-dot-plots"
driver.get(url)


In [ ]:

# data = requests.post('https://api.track.toggl.com/api/v9/me/sessions', headers={'content-type': 'application/json'})
# print(data.json())